In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./part.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Vehicle_Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(666):
            try:
                if url == '':
                    status = 'ok'
                    
                    break
                    
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_tr = html.xpath('//tr[@class="detail-app-row"]')
                
                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                while True:
                    for tr in list_tr:
                        year = int(tr.xpath('./td/text()')[0].strip())
                        make = tr.xpath('./td/text()')[1].strip()
                        model = tr.xpath('./td/text()')[2].strip()
                        engine = tr.xpath('./td/text()')[3].strip()

                        if make not in dict_vehicle:
                            dict_vehicle[make] = {model: {engine: [year]}}
                        else:
                            if model not in dict_vehicle[make]:
                                dict_vehicle[make][model] = {engine: [year]}
                            else:
                                if engine not in dict_vehicle[make][model]:
                                    dict_vehicle[make][model][engine] = [year]
                                else:
                                    dict_vehicle[make][model][engine].append(year)
                            
                    # = = = = = = = = = = = = = = =
                    
                    if len(html.xpath('//a[@aria-label="Next"]')) == 0:
                        break
                    else:
                        resp = requests.get('https://www.dormanproducts.com' + html.xpath('//a[@aria-label="Next"]/@href')[0].strip(),
                                            headers=UA.get_User_Agent_Requests(),
                                            proxies=Proxy.get_Proxy_Requests()).text
                        soup = BeautifulSoup(resp, 'lxml')
                        html = etree.HTML(str(soup))
                        list_tr = html.xpath('//tr[@class="detail-app-row"]')
                        
                # = = = = = = = = = = = = = = =
                        
                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        for engine in dict_vehicle[make][model]: 
                            year_max = max(dict_vehicle[make][model][engine])
                            year_min = min(dict_vehicle[make][model][engine])
                            if year_max == year_min:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine)
                            else:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Vehicle_Url': url,
                                         'Vehicle': vehicle}])
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Vehicle_Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./vehicle_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：1117



Traceback (most recent call last):
  File "src/gevent/greenlet.py", line 908, in gevent._gevent_cgreenlet.Greenlet.run
  File "/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_1290/2516154702.py", line 143, in crawler
    df = pd.concat([df, df_temp], ignore_index=True).fillna('')
                        ^^^^^^^
UnboundLocalError: cannot access local variable 'df_temp' where it is not associated with a value
2024-01-05T00:41:49Z <Greenlet at 0x11f7f04a0: crawler> failed with UnboundLocalError

Traceback (most recent call last):
  File "src/gevent/greenlet.py", line 908, in gevent._gevent_cgreenlet.Greenlet.run
  File "/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_1290/2516154702.py", line 143, in crawler
    df = pd.concat([df, df_temp], ignore_index=True).fillna('')
                        ^^^^^^^
UnboundLocalError: cannot access local variable 'df_temp' where it is not associated with a value
2024-01-05T00:41:49Z <Greenlet at 0x11f469940: crawler> failed with U

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=477&PartType=Engine%20Oil%20Drain%20Plug  <->  [ok] - 剩余数量：1097 - 当前时间：08:41:53
  <->  [ok] - 剩余数量：1096 - 当前时间：08:41:53
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=21570&PartType=Engine%20Oil%20Drain%20Plug  <->  [ok] - 剩余数量：1095 - 当前时间：08:41:54
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=102404&PartType=Engine%20Oil%20Drain%20Plug  <->  [ok] - 剩余数量：1094 - 当前时间：08:41:54
  <->  [ok] - 剩余数量：1093 - 当前时间：08:41:54
  <->  [ok] - 剩余数量：1092 - 当前时间：08:41:54
  <->  [ok] - 剩余数量：1091 - 当前时间：08:41:54
  <->  [ok] - 剩余数量：1090 - 当前时间：08:41:54
  <->  [ok] - 剩余数量：1089 - 当前时间：08:41:54
  <->  [ok] - 剩余数量：1088 - 当前时间：08:41:54
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=128356&PartType=Engine%20Oil%20Dipstick%20Flange%20Repair%20Kit  <->  [ok] - 剩余数量：1087 - 当前时间：08:41:56
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=73584&PartType=Engine%20Oil%20Dipstick%20Flange%20Repair%20Kit  <->  [ok

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=1110&PartType=Engine%20Oil%20Drain%20Plug  <->  [ok] - 剩余数量：1025 - 当前时间：08:46:15
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=102290&PartType=Engine%20Oil%20Drain%20Plug  <->  [ok] - 剩余数量：1024 - 当前时间：08:46:15
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=26497&PartType=Engine%20Oil%20Drain%20Plug  <->  [ok] - 剩余数量：1023 - 当前时间：08:46:17
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=17459&PartType=Engine%20Oil%20Drain%20Plug  <->  [ok] - 剩余数量：1022 - 当前时间：08:46:20
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=131&PartType=Engine%20Oil%20Drain%20Plug  <->  [ok] - 剩余数量：1021 - 当前时间：08:46:22
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=25744&PartType=Engine%20Oil%20Drain%20Plug  <->  [ok] - 剩余数量：1020 - 当前时间：08:46:24
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=21569&PartType=Engine%20Oil%20Drain%20Plug  <->  [ok] - 剩余数量：1019 - 当前时间：08:46:28
https://

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=15340&PartType=Engine%20Oil%20Drain%20Plug  <->  [ok] - 剩余数量：953 - 当前时间：08:48:27
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=362&PartType=Engine%20Oil%20Drain%20Plug  <->  [ok] - 剩余数量：952 - 当前时间：08:48:27
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=102324&PartType=Engine%20Oil%20Drain%20Plug  <->  [ok] - 剩余数量：951 - 当前时间：08:48:28
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=102289&PartType=Engine%20Oil%20Drain%20Plug  <->  [ok] - 剩余数量：950 - 当前时间：08:48:44
  <->  [ok] - 剩余数量：949 - 当前时间：08:48:44
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=140&PartType=Engine%20Oil%20Drain%20Plug  <->  [ok] - 剩余数量：948 - 当前时间：08:48:45
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=17464&PartType=Engine%20Oil%20Drain%20Plug  <->  [ok] - 剩余数量：947 - 当前时间：08:48:50
  <->  [ok] - 剩余数量：946 - 当前时间：08:48:50
  <->  [ok] - 剩余数量：945 - 当前时间：08:48:50
  <->  [ok] - 剩余数量：944 - 当前时间：08:48:50


...

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=77887&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：284 - 当前时间：09:09:03
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=77883&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：283 - 当前时间：09:09:03
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=72775&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：282 - 当前时间：09:09:03
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=72632&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：281 - 当前时间：09:09:03
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=133873&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：280 - 当前时间：09:09:04
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=133875&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：279 - 当前时间：09:09:05
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=133874&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：278 - 当前时间：09:09:05
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=133876&PartTy

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=77641&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：222 - 当前时间：09:09:44
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=26843&PartType=Engine%20Oil%20Drain%20Plug%20Gasket  <->  [ok] - 剩余数量：221 - 当前时间：09:09:45
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=78782&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：220 - 当前时间：09:09:46
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=76316&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：219 - 当前时间：09:09:46
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=77878&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：218 - 当前时间：09:09:47
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=78477&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：217 - 当前时间：09:09:48
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=77888&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：216 - 当前时间：09:09:48
https://www.dormanproducts.com/itemdetailapp.aspx?ProductI

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=86283&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：160 - 当前时间：09:11:01
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=106492&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：159 - 当前时间：09:11:01
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=133744&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：158 - 当前时间：09:11:01
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=86461&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：157 - 当前时间：09:11:01
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=106823&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：156 - 当前时间：09:11:02
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=85673&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：155 - 当前时间：09:11:02
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=106079&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：154 - 当前时间：09:11:03
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=85959&PartTy

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=131736&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：98 - 当前时间：09:11:29
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=107488&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：97 - 当前时间：09:11:30
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=133918&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：96 - 当前时间：09:11:30
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=106081&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：95 - 当前时间：09:11:32
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=133919&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：94 - 当前时间：09:11:32
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=132810&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：93 - 当前时间：09:11:32
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=126743&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：92 - 当前时间：09:11:33
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=106035&PartType=

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=49839&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：34 - 当前时间：09:12:21
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=26984&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：33 - 当前时间：09:12:22
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=135569&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：32 - 当前时间：09:12:22
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=135572&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：31 - 当前时间：09:12:23
  <->  [ok] - 剩余数量：30 - 当前时间：09:12:23
  <->  [ok] - 剩余数量：29 - 当前时间：09:12:23
  <->  [ok] - 剩余数量：28 - 当前时间：09:12:23
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=136979&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：27 - 当前时间：09:12:24
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=135571&PartType=Engine%20Oil%20Pan  <->  [ok] - 剩余数量：26 - 当前时间：09:12:24
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=133965&PartType=Engine%20Oil%20Pan%2